<a href="https://colab.research.google.com/github/sunflowcoco/SS_Classification/blob/master/CNN_VPN_320.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
'''
SS 320

自己的方法实现



'''

from __future__ import print_function
import keras
import sys
from keras.datasets import mnist

from keras.models import Sequential

from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop


from keras import backend as K

import numpy as np  
import os
import time

from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score,accuracy_score

np.random.seed(1337)
from keras.utils import np_utils  
from keras.models import Model

#DNN setting
batch_size = 256
num_classes = 6
epochs = 100
#input image dimensions
# img_rows, img_cols = 20, 16
img_rows, img_cols = 8, 40



###############################################################################
#If equall to 'yes', then load from Bin file
FAST_LOAD = 'no'
#postfix
#FILE_TYPE = '21_40d_62851r_122010'#'756d_less'#'1134d'
FILE_TYPE='21_40d_104685r_12316_VPN'
#Feature column
COLUMN = 320
#K-Fold
K = 10
k = 1
# if len(sys.argv)>=2:
# 	k = int(sys.argv[1])
# if len(sys.argv)>=3:
# 	K = int(sys.argv[2])	


path = '/content/drive/SS/'

#Log setting
LOG_FLAG = 1
log_str = ''
# logfilename = '../log/'+os.path.basename(sys.argv[0])+'_'+str(K)+'_'+str(k)+'.txt'
logfilename = path+ '/log/'+os.path.basename(sys.argv[0])+'_'+str(K)+'_'+str(k)+'.txt'

log_str = log_str + 'Start: '+time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))+'\r\n'
	
# src_file = '../data/sample_combinnativeflat21_40d_134685r_12522.txt'
# bin_file = '../data/sample_combinnativeflat21_40d_134685r_320.bin'

# src_file =path + 'data/sample_combinnativeflat21_40d_134685r_12522.txt'
# bin_file=path + 'data/sample_combinnativeflat21_40d_104685r_800.bin'

# src_file =path + 'data/sample_combinnativeflat21_40d_104685r_12711.txt'
# bin_file=path + 'data/sample_combinnativeflat21_40d_104685r_320.bin'

src_file =path + 'data/sample_combinnativeflat21_40d_5112r_12316_VPN.txt'
bin_file=path + 'data/sample_combinnativeflat21_40d_5112r_12316_VPN.bin'


sample = []
if FAST_LOAD=='yes':
	sample = np.fromfile(bin_file, dtype=np.double) 
	sample = sample.reshape(-1, COLUMN+1)
else:
	orig = (np.loadtxt(src_file, dtype=np.str, delimiter=","))[:,40:360].astype(np.double).reshape(-1, 320)
	label = (np.loadtxt(src_file, dtype=np.str, delimiter=","))[:,-1].astype(np.double).reshape(-1, 1)-1
	
	print(orig.shape)
	print(label.shape)
	
	sample = np.hstack((orig, label))
	np.random.shuffle(sample)
# 	sample.tofile(bin_file)



gridLen = int(len(sample)*1.0/K)
s_index = (k-1)*gridLen
e_index = k*gridLen

testX = sample[s_index:e_index, :-1]*255
testY = sample[s_index:e_index, -1]


if k==1:
	trainX = sample[e_index:, :-1]*255
	trainY = sample[e_index:, -1]
elif k==K:
	trainX = sample[0:s_index, :-1]*255
	trainY = sample[0:s_index, -1]
else:	
	trainX = np.vstack((sample[0:s_index, :-1], sample[e_index:, :-1]))*255
	trainY = np.hstack((sample[0:s_index, -1], sample[e_index:, -1]))

trainYBackup = trainY
testYBackup = testY

print(trainX.max())
print(trainX.min())

del sample

print ('Train Size:'+str(len(trainY)))
print ('Test Size:'+str(len(testY)))

print('trainX.shape:' ,trainX.shape)
print('testX.shape:' , testX.shape)
# print(trainX[0])


trainX = trainX.reshape(-1, img_rows, img_cols, 1)
testX = testX.reshape(-1, img_rows, img_cols, 1)
print('after reshape')
print('trainX.shape:' , trainX.shape)
# print(trainX[0])

img_cols= img_cols-0
trainX = trainX[:,:,0:img_cols]
testX = testX[:,:,0:img_cols]



trainX_T=np.zeros((len(trainX),img_cols,img_rows))
testX_T=np.zeros((len(testX),img_cols,img_rows))

for i in range(len(trainX)):
	trainX_T[i,:,:]=np.transpose(trainX[i,:,:])
  
for i in range(len(testX)):
	testX_T[i,:,:]=np.transpose(testX[i,:,:])  
trainX_T = trainX_T.reshape(-1, img_cols,img_rows, 1)  
testX_T = testX_T.reshape(-1, img_cols,img_rows, 1)

print('trainX_T.shape:',trainX_T.shape)
# print(trainX_T[0])




(5112, 320)
(5112, 1)
255.0
0.0
Train Size:4601
Test Size:511
trainX.shape: (4601, 320)
testX.shape: (511, 320)
after reshape
trainX.shape: (4601, 8, 40, 1)
trainX_T.shape: (4601, 40, 8, 1)


In [31]:

trainY=np_utils.to_categorical(trainY, num_classes)
testY=np_utils.to_categorical(testY, num_classes)

# input_shape = (img_rows, img_cols, 1)

input_shape = (img_cols, img_rows, 1)

from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath='bestmodel_yyw_320_20.h5',monitor='val_loss',verbose=1,save_best_only=True)


input_data = Input(shape=input_shape)
conv = Conv2D(32, kernel_size=(3, 3), activation='relu')(input_data)
# conv = MaxPooling2D(pool_size=(2, 2))(conv)
conv = Conv2D(64, (5, 5), activation='relu')(conv)
conv = MaxPooling2D(pool_size=(2, 2))(conv)
conv = Flatten()(conv)
conv_output = Dense(128, activation='relu')(conv)
full = Dense(64, activation='relu')(conv_output)
full = Dropout(0.4)(full)
full_output = Dense(num_classes, activation='softmax')(full)

model = Model(inputs=input_data, outputs=full_output)


model.summary()

# model.compile(loss=keras.losses.categorical_crossentropy,
# 			  optimizer=keras.optimizers.Adadelta(),
# 			  metrics=['accuracy'])
rmsprop=RMSprop(lr=0.002,rho=0.9,epsilon=1e-08,decay=0.00) 
model.compile(loss=keras.losses.categorical_crossentropy,
			  optimizer=rmsprop,
			  metrics=['accuracy'])

history=model.fit(trainX_T, trainY,
		  batch_size=batch_size,
		  epochs=epochs,
		  verbose=1,
		  validation_data=(testX_T, testY),
		  callbacks=[checkpoint])


#model.save(os.path.basename(sys.argv[0])+'_model'+'_'+FILE_TYPE+'.h5')
test_rst = model.predict(testX_T)

test_label = np.argmax(test_rst, axis=1).astype(np.int32)
#confusion_matrix = np.zeros((num_classes,num_classes))
testY = testYBackup.astype(np.int32)

print(history.history,file=open('history_320','w'))
cf=confusion_matrix(testY,test_label)


pre=precision_score(testY,test_label,average=None)
rec=recall_score(testY,test_label,average=None)
f1_s=f1_score(testY,test_label,average=None)

# print(cf,file=open('confusion_matrix_320','w'))
# print(pre,file=open('precision_320','w'))
# print(rec,file=open('recall_320','w'))
# print(f1_s,file=open('f1_score_320','w'))

print('packet num =', img_cols)
print(cf)
print(pre)
print(rec)
print(f1_s)

acc=accuracy_score(testY,test_label)
print(acc)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 40, 8, 1)          0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 38, 6, 32)         320       
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 34, 2, 64)         51264     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 17, 1, 64)         0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 1088)              0         
_________________________________________________________________
dense_31 (Dense)             (None, 128)               139392    
_________________________________________________________________
dense_32 (Dense)             (None, 64)                8256      
__________